In [146]:
# Importing required libraries
import pandas as pd
import seaborn as sns

In [147]:
# Importing the google api library
from googleapiclient.discovery import build

In [ ]:
# Generate an API key from Google Developer Console and replace xxx with it
api_key = 'xxx'
channel_ids = ['UCzOVIHSg-GeenFlGHttWQwA',
               'UCaO19EP1T3m61v7lhXO0Cfg',
               'UCoxPghkIDTPxtgZV4Zx8tMw'
              ]

youtube = build('youtube', 'v3', developerKey = api_key)

## Function to get channel statistics

In [ ]:
def get_channel_stats(youtube, channel_id):
    all_data = []
    request = youtube.channels().list(
        part = 'snippet, contentDetails, statistics',
        # Channel IDs have to be comma-seperated
        id = ','.join(channel_ids))
    response = request.execute()

    # using for loop to get the data for all the channels
    for i in range(len(response['items'])):
        data = dict(Channel_name = response['items'][i]['snippet']['title'],
                   Subscribers = response['items'][i]['statistics']['subscriberCount'],
                   Views = response['items'][i]['statistics']['viewCount'],
                   Total_videos = response['items'][i]['statistics']['videoCount'],
                   playlist_id = response['items'][i]['contentDetails']['relatedPlaylists']['uploads']
                   )
        all_data.append(data)

    return all_data

In [ ]:
channel_statistics = get_channel_stats(youtube, channel_ids)

In [ ]:
channel_data = pd.DataFrame(channel_statistics)

In [ ]:
channel_data

In [ ]:
channel_data['Subscribers'] = pd.to_numeric(channel_data['Subscribers'])
channel_data['Views'] = pd.to_numeric(channel_data['Views'])
channel_data['Total_videos'] = pd.to_numeric(channel_data['Total_videos'])
channel_data.dtypes

In [ ]:
# Visualizing the key parameters
ax = sns.barplot(x = 'Channel_name', y = 'Subscribers', data  = channel_data)

In [ ]:
ax = sns.barplot(x = 'Channel_name', y = 'Views', data  = channel_data)

In [ ]:
ax = sns.barplot(x = 'Channel_name', y = 'Total_videos', data  = channel_data)

## Function to get the video IDs

In [ ]:
channel_data

In [ ]:
playlist_id = channel_data.loc[channel_data['Channel_name'] == 'yatri doctor', 'playlist_id'].iloc[0]

In [ ]:
def get_video_ids(youtube, playlist_id):
    request = youtube.playlistItems().list(
        part = 'contentDetails',
        playlistId = playlist_id,
        maxResults = 50)
    response = request.execute()

    video_ids = []
    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])

    next_page_token = response.get('nextPageToken')
    # get is used to avoid the case where there is no next_page_token
    more_pages = True

    # Below is the logic to fetch all the video ids
    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.playlistItems().list(
                part = 'contentDetails',
                playlistId = playlist_id,
                maxResults = 50,
                pageToken = next_page_token
            )
            response = request.execute()
            
            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])

            next_page_token = response.get('nextPageToken')

    return video_ids

In [ ]:
video_ids = get_video_ids(youtube, playlist_id)

## Function to get video details

In [ ]:
def get_video_details(youtube, video_ids):
    all_video_stats = []
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part = 'snippet, statistics',
            # id of video has to be comma seperated
            # also slicing is done as the API request accepts only 50 ids at once
            id = ','.join(video_ids[i:i+50])
        )
        response = request.execute()
        for video in response['items']:
            video_stats = dict(Title = video['snippet']['title'],
                               Published_date = video['snippet']['publishedAt'],
                               Views = video['statistics']['viewCount'],
                               Likes = video['statistics']['likeCount'],
                               Comments = video['statistics']['commentCount']
                              )
            all_video_stats.append(video_stats)

    return all_video_stats

In [ ]:
video_details = get_video_details(youtube, video_ids)

In [ ]:
video_data = pd.DataFrame(video_details)

In [ ]:
# Converting the info into relevant data types
video_data['Published_date'] = pd.to_datetime(video_data['Published_date']).dt.date
video_data['Views'] = pd.to_numeric(video_data['Views'])
video_data['Likes'] = pd.to_numeric(video_data['Likes'])
video_data['Comments'] = pd.to_numeric(video_data['Comments'])
video_data

In [ ]:
# Finding the top 10 most viewed videos
top10_videos = video_data.sort_values(by = 'Views', ascending = False).head(10)

In [ ]:
top10_videos

In [ ]:
# Analyzing Views Against Title
ax1 = sns.barplot(x = 'Views', y = 'Title', data = top10_videos)

In [ ]:
# Analyzing data with respect to month

# Segregating data by month
video_data['Month'] = pd.to_datetime(video_data['Published_date']).dt.strftime('%b')

In [ ]:
video_data

In [ ]:
videos_per_month = video_data.groupby('Month', as_index=False).size()

In [ ]:
videos_per_month

In [ ]:
# Sorting w.r.t Month
sort_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'June', 'July', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [ ]:
videos_per_month.index = pd.CategoricalIndex(videos_per_month['Month'], categories = sort_order, ordered = True)

In [ ]:
videos_per_month = videos_per_month.sort_index()

In [ ]:
ax2 = sns.barplot(x = 'Month', y = 'size', data = videos_per_month)